In [ ]:
import csv
import time
from typing import List, Dict

from blazemetrics import (
    compute_text_metrics,
    aggregate_samples,
    MetricsExporters,
    set_parallel, set_parallel_threshold, get_parallel, get_parallel_threshold,
)

print("✅ Imports ready")

# Tune parallelism (example defaults)
set_parallel(True)
set_parallel_threshold(512)
print("Parallel:", get_parallel(), "Threshold:", get_parallel_threshold())


In [ ]:
# Example: epoch loop with CSV export

def generate_predictions(prompts: List[str], epoch: int) -> List[str]:
    # replace with a real model call
    return [p.replace("t", "t").strip() for p in prompts]

prompts = [f"t{i} t{i+1} t{i+2}" for i in range(100)]
references = [[f"t{i} t{i+1} t{i+3}"] for i in range(100)]

exporters = MetricsExporters(prometheus_gateway=None, statsd_addr=None)

with open("training_metrics.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["epoch", "metric", "value"])

    for epoch in range(1, 6):
        t0 = time.perf_counter()
        preds = generate_predictions(prompts, epoch)
        sample_metrics = compute_text_metrics(preds, references)
        agg = aggregate_samples(sample_metrics)

        # write CSV
        for k, v in agg.items():
            writer.writerow([epoch, k, f"{v:.4f}"])

        # export to metrics backends
        exporters.export(agg, labels={"epoch": str(epoch)})

        t1 = time.perf_counter()
        print(f"Epoch {epoch}: computed metrics in {(t1-t0)*1000:.1f} ms")
        print({k: round(v, 4) for k, v in agg.items()})

print("✅ Wrote training_metrics.csv and exported metrics (mock)")
